In [1]:
# import package
import pandas as pd
import numpy as np
import time as tm

from selenium import webdriver

import re
import os

from selenium.common.exceptions import StaleElementReferenceException

In [2]:
import numpy as np
import pandas as pd
import nltk
from gensim.summarization import summarize
from sumy.utils import get_stop_words
from sumy.nlp.stemmers import Stemmer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer as sumytoken
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.utils import get_stop_words
from sumy.nlp.stemmers import Stemmer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer as sumytoken
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.utils import get_stop_words
from sumy.nlp.stemmers import Stemmer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer as sumytoken
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.nlp.stemmers import Stemmer
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
LANGUAGE = "english"
stemmer = Stemmer(LANGUAGE)

def lexrank_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string((text), sumytoken(LANGUAGE))
    summarizer_LexRank = LexRankSummarizer(stemmer)
    summarizer_LexRank.stop_words = get_stop_words(LANGUAGE)
    sentences = []
    for sentence in summarizer_LexRank(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

[nltk_data] Downloading package punkt to C:\Users\Muneeb
[nltk_data]     Sial\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def makeDriver(url):
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    
    try:
        ########for jupyternotebook#######
        driver = webdriver.Chrome(options = chrome_options, executable_path = (os.getcwd() + '\\chromedriver'))
        
        driver.get(url)
        
    except:
        
        print ("Error in making DRIVER")
        driver=None

    return driver

#############################################################################################################

def privacyPolicyURL(atag_text):
    
    for i in atag_text:
        
        href = i.get_attribute("href")

        if (href is not None and "privacy" in href):
            
            return (href)
        
    return None

def scrap_policy(site_name):
            
    print ("http://www." + site_name)
    driver = makeDriver ("http://www." + site_name)

    if driver is not None:

        websitename = "www." + site_name
        atag_text = driver.find_elements_by_css_selector("a")

        try:
            href = privacyPolicyURL(atag_text)
        except StaleElementReferenceException:
            print ("stealing error")

        if href is not None:

            driver = makeDriver (href)
            if driver is not None:

                #print ("------------------------------Extracting POLICY---------------------------------------")
                privacy_policy = driver.find_element_by_tag_name("body").text
                
                if len(privacy_policy) > 0:

                    #print ("-----------------------------------#Succesful#---------------------------------------------")
                    #print (privacy_policy)
                    return privacy_policy

                else:
                    return "Oops something went wrong. Please try again"
            else:
                return "Oops something went wrong. Please try again"
        else:
            return "Oops something went wrong. Please try again"    
    else:
        return "Oops something went wrong. Please try again"

In [4]:
def generate(typpe, msg):
   
    p_policy = ""
    if typpe == "search":
        p_policy = scrap_policy(msg)
    elif typpe == "a_summary":
        pp_smry = _generate_summary(msg)
        p_policy = msg
        return p_policy , pp_smry
    else:
        p_policy = msg
    pp_smry = p_policy

    if p_policy != "Oops something went wrong. Please try again":
        pp_list = p_policy.split('\n')
        p_policy = ""

        for pp in pp_list:
            if len(pp) < 250:
                continue
            else:
                p_policy += pp + '\n'

        pp_list = p_policy.split('\n')
        pp_smry = ""

        for pp in pp_list:
            SENTENCES_COUNT = len(sent_tokenize(pp))/3
            if SENTENCES_COUNT == 0:
                SENTENCES_COUNT += 1
            try:
                smry = lexrank_summarizer(pp, stemmer, LANGUAGE, SENTENCES_COUNT)
                if smry != "":
                    pp_smry += "\t===>\t" + smry + "\n\n"
            except:
                pp_smry += ""

    return p_policy , pp_smry

In [5]:
#policy , summary = generate("search", "instagram.com")
#print ("-----------------------------------------policy--------------------------------------------------")
#print (policy)
#print ("-----------------------------------------summary-------------------------------------------------")
#print (summary)

In [6]:
import re
from eazymind.nlp.eazysum import Summarizer
key = "feb3505f05d9a114a1167b3263ea8d48"
cleaner = Summarizer(key)
from gensim.summarization.summarizer import summarize
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize





def _generate_summary(text):
    
    text = text.replace('\t===>\t','')
    text = text.replace('\n\n','')
    
    print (text)

    sentences = sent_tokenize(text)

    sentences_list = []
    sentence = ""

    for sen in sentences:
        if len(sentence) + len(sen) > 400:
            sentences_list.append(sentence)
            sentence = ""
        sentence += sen
    sentences_list.append(sentence)
    
    print ('sentences = ', len(sentences_list))
    
    cleaned_text = ""
    i = 1
    
    for sen in sentences_list:
        try:
            t_cleaned = cleaner.run(sen)
        except:
            continue
        if t_cleaned.find('<!DOCTYPE html>', 0, 15) == -1:
            cleaned_text += ('\t===>\t' + t_cleaned + '\n\n')
            #print ("SENTENCE")
            #print (sen)
            #print ("SUMMARY")
            #print (t_cleaned)
        else:
            t_cleaned = lexrank_summarizer(sen, stemmer, LANGUAGE, 1)
            if t_cleaned != "":
                cleaned_text += "\t===>\t" + t_cleaned + "\n\n"
        
    
    print (len(cleaned_text))
    
    if len(cleaned_text) != 0:
        print ('!=0')
        return cleaned_text
    return "Oops something went wrong. Please try again"

In [ ]:
import socket
from _thread import *
import threading
import json
from io import StringIO
import time

#from ftfy import fix_text


def threaded(c):
    
    time.sleep(1)
    data = c.recv(1000000)
    time.sleep(1)
    print (data)
    
    #data = fix_text(data)
    data = json.loads( data )
    message = data["message"]
    typee = data['type']

    policy, smry = generate(typee, message)
    dt = json.dumps({"policy": policy, "smry": smry}).encode('utf-8')
    dt += b'\n'
    print (dt)
    c.send( dt )
    
    c.close()
port = 5001
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(( "0.0.0.0" , port ))
print("socket binded to port", port )
s.listen(1)
print("socket is listening")
all_conn = []
while True:
    conn, addr = s.accept()
    print('Connected to :', addr[0], ':', addr[1])
    all_conn.append( conn )
    start_new_thread(threaded, (conn,))


socket binded to port 5001
socket is listening
Connected to : 192.168.0.100 : 48394
b'{"type":"search","message":"airbnb.com"}\n'
http://www.airbnb.com
b'{"policy": "Thank you for using Airbnb! Your trust is important to us and we\\u2019re committed to protecting the privacy and security of your personal information. The information that\\u2019s shared with us helps us to provide a great experience with Airbnb. We have a dedicated privacy team that\\u2019s committed to protecting all the personal information we collect and help ensure that personal information is handled properly worldwide.\\nThis Privacy Policy describes how we collect, use, process, and disclose your personal information, in conjunction with your access to and use of the Airbnb Platform and the Payment Services. This privacy policy describes our privacy practices for all websites, platforms and services that link to it. Please read the privacy policy on the applicable site.\\nIf your country of residence is outside o

Connected to : 192.168.0.100 : 48412
b'{"type":"a_summary","message":"\\t===>\\tThank you for using Airbnb!\\n\\n\\t===>\\tThis Privacy Policy describes how we collect, use, process, and disclose your personal information, in conjunction with your access to and use of the Airbnb Platform and the Payment Services.\\n\\n\\t===>\\tWe ask for and collect the following personal information about you when you use the Airbnb Platform.\\n\\n\\t===>\\tTo help create and maintain a trusted environment, we may collect identity verification information (such as images of your government issued ID, passport, national ID card, or driving license, as permitted by applicable laws) or other authentication information.\\n\\n\\t===>\\tAdditional Profile Information.\\n\\n\\t===>\\tGeo-location Information.\\n\\n\\t===>\\tLog Data and Device Information.\\n\\n\\t===>\\tWe use cookies and other similar technologies when you use our platform, use our mobile app, or engage with our online ads or email commun

997
!=0
b'{"policy": "\\t===>\\tThank you for using Airbnb!\\n\\n\\t===>\\tThis Privacy Policy describes how we collect, use, process, and disclose your personal information, in conjunction with your access to and use of the Airbnb Platform and the Payment Services.\\n\\n\\t===>\\tWe ask for and collect the following personal information about you when you use the Airbnb Platform.\\n\\n\\t===>\\tTo help create and maintain a trusted environment, we may collect identity verification information (such as images of your government issued ID, passport, national ID card, or driving license, as permitted by applicable laws) or other authentication information.\\n\\n\\t===>\\tAdditional Profile Information.\\n\\n\\t===>\\tGeo-location Information.\\n\\n\\t===>\\tLog Data and Device Information.\\n\\n\\t===>\\tWe use cookies and other similar technologies when you use our platform, use our mobile app, or engage with our online ads or email communications. In many cases the information we colle

Connected to : 192.168.0.100 : 48448
b'{"type":"search","message":"whatsapp.com"}\n'
http://www.whatsapp.com
b'{"policy": "Our mission is to connect the world privately by designing a product that\'s simple and private. So whether you\'re sending a message to your loved ones, or video calling a friend, your communications remain secure and you\'re always in control. Here, your conversations stay just between you.\\nWe want to be clear about what information remains private and what information we collect and share with our parent company, Facebook. The information we share helps us provide the best user experience and improve security. For the most up-to-date and accurate information, see our privacy policy.\\n", "smry": "\\t===>\\tOur mission is to connect the world privately by designing a product that\'s simple and private.\\n\\n\\t===>\\tWe want to be clear about what information remains private and what information we collect and share with our parent company, Facebook.\\n\\n"}\n'

b'{"policy": "We joined Facebook in 2014. WhatsApp is now part of the Facebook family of companies. Our Privacy Policy explains how we work together to improve our services and offerings, like fighting spam across apps, making product suggestions, and showing relevant offers and ads on Facebook. Nothing you share on WhatsApp, including your messages, photos, and account information, will be shared onto Facebook or any of our other family of apps for others to see, and nothing you post on those apps will be shared on WhatsApp for others to see.\\nYour messages are yours, and we can\\u2019t read them. We\\u2019ve built privacy, end-to-end encryption, and other security features into WhatsApp. We don\\u2019t store your messages once they\\u2019ve been delivered. When they are end-to-end encrypted, we and third parties can\\u2019t read them.\\nNew ways to use WhatsApp. We will explore ways for you and businesses to communicate with each other using WhatsApp, such as through order, transact